# Example usage

To use `ghgql` in a project:

I've designed `ghgql` to work on query files rather than pure query strings. This is fully intentional to not get into the habit of putting query strings all around in the code. Instead when the queries live in files we can validate them agains Github's GraphQL schema.

Long story short, for the purpose of this demo we need a `get_query_as_file` function that converts a graphql string to a graphql file with the same content:

In [1]:
from contextlib import contextmanager
from tempfile import NamedTemporaryFile

@contextmanager
def get_query_as_file(query: str) -> str:
    """
    Returns a temporary filename with the given query written to it.
    Use this as a context manager:

        with get_query_as_file(query="query {}") as filename:
            ghapi query_from_file(filename)
    """
    file_handle = NamedTemporaryFile(mode="w+", encoding="utf-8")
    file_handle.writelines(query)
    file_handle.flush()
    try:
        yield file_handle.name
    finally:
        file_handle.close()

Then we can convert a query like this:

In [2]:
query = """
query ($searchQuery: String!) {
  search(query: $searchQuery, type: ISSUE, first: 100) {
    edges {
      node {
        ... on Issue {
          id
          number
          title
          url
        }
      }
    }
  }
}
"""


and use it as a context manager to query for the first issue in the `llvm/llvm-project` repository:

In [3]:
import os
from pprint import pprint
import ghgql

with ghgql.GithubGraphQL(token=os.getenv("GITHUB_TOKEN")) as ghapi:
    with get_query_as_file(query) as filename:
      result = ghapi.query_from_file(filename=filename, variables={"searchQuery": "llvm/llvm-project"})
      pprint(result.get("search.edges"))

[{'node': {'id': 'I_kwDOHicqdc5RG-tC',
           'number': 16,
           'title': 'llvm/llvm-project',
           'url': 'https://github.com/KhushP786/open-sauced-goals/issues/16'}},
 {'node': {}},
 {'node': {}},
 {'node': {}},
 {'node': {'id': 'I_kwDOHicqdc5RG-on',
           'number': 15,
           'title': 'apple/llvm-project',
           'url': 'https://github.com/KhushP786/open-sauced-goals/issues/15'}},
 {'node': {'id': 'I_kwDOHO0h_s5QNj9-',
           'number': 9,
           'title': 'git clone https://github.com/llvm/llvm-project.git',
           'url': 'https://github.com/OsvaldoRino/os/issues/9'}},
 {'node': {}},
 {'node': {}},
 {'node': {'id': 'I_kwDODI9FZc5QcrSZ',
           'number': 74505,
           'title': 'Add source-build leg to llvm-project',
           'url': 'https://github.com/dotnet/runtime/issues/74505'}},
 {'node': {}},
 {'node': {}},
 {'node': {'id': 'I_kwDOEd-da85RliO5',
           'number': 3019,
           'title': 'Support Zstd-compressed ELF debug inf

I know this looks a bit weird but usually when you have your queries stored in files you can get rid of the inner `with`-context. 